## SQL Queries - Case Study for Muse group
---
Candidate: **Nicolas Frate**

In [1]:
# Loading configurations
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from functools import reduce
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import time 
from datetime import datetime, date, timedelta


app_name = "MUSE SQL"

# setup spark configs
conf = SparkConf().setAppName(app_name)\
    .set("spark.dynamicAllocation.maxExecutors", 143)\
    .set('spark.rpc.message.maxSize','256')\
    .set("spark.sql.orc.impl", "native")\
    .set("spark.sql.hive.convertMetastoreOrc", "true")\
    .set("spark.sql.orc.enableVectorizedReader", "true")\
    .set("spark.sql.execution.arrow.pyspark.enabled", "false")\
    .set("spark.sql.sources.partitionOverwriteMode","dynamic")\
    .set("spark.sql.qubole.directWrites.dataSourceDynamicParitionOverwrite.enabled", "true")\
    .set("spark.sql.qubole.directWrites.dynamicPartitionOverwrite.enabled", "true")\
    .set("spark.sql.repl.eagerEval.enabled", "true")\
    .set('spark.sql.legacy.timeParserPolicy','LEGACY')\
    .set("hive.exec.dynamic.partition", "true")\
    .set("hive.exec.dynamic.partition.mode", "nonstrict")\
    .set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")\
    .set("spark.sql.catalog.spark_catalog.type", "hive")\
    .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    
sc = SparkContext(conf = conf)

spark = SparkSession.builder.enableHiveSupport()\
        .config(conf=conf)\
        .getOrCreate()

In [2]:
events = spark.createDataFrame(pd.read_csv('Events.csv')).dropna()
sales = spark.createDataFrame(pd.read_csv('Sales.csv')).dropna()

events.createOrReplaceTempView('events_data')
sales.createOrReplaceTempView('sales_data')

### Build an event funnel for the data via query
---

In [3]:
%%sparksql

select
eventname
,sum(sessions) as sessions
,sum(totalusers) as totalusers
from events_data
group by 1
order by 2 desc

eventname,sessions,totalusers
page_view,3317710.0,3022549.0
view_item,3112322.0,2912252.0
subcription_view,160363.0,157267.0
add_to_cart,153734.0,148780.0
purchase,104873.0,102905.0


### Separate by Pass and OTP
---

In [4]:
%%sparksql -l 30

select
purchaseType
,eventname

,sum(sessions) as sessions
,sum(totalusers) as totalusers

from events_data
group by 1,2
order by 1,3 desc

purchaseType,eventname,sessions,totalusers
NaN,page_view,3317710.0,3022549.0
NaN,view_item,3112322.0,2912252.0
OTP,add_to_cart,153734.0,148780.0
OTP,purchase,92307.0,90343.0
Subscription,subcription_view,160363.0,157267.0
Subscription,purchase,12566.0,12562.0


Conversion Rate by Purchase type

In [5]:
%%sparksql -l 30

with data as (
    select
    purchaseType
    ,eventname

    ,sum(sessions) as sessions
    ,sum(totalusers) as totalusers

    from events_data
    group by 1,2
    order by 1,3 desc
)

select
purchaseType 
,sum(case when eventname in ('purchase') then totalusers end) / sum(case when eventname in ('add_to_cart', 'subcription_view') then totalusers end) as conversion_rate
from data
where purchaseType != 'NaN'
group by 1

purchaseType,conversion_rate
Subscription,0.07987689725117157
OTP,0.6072254335260115


### How should the funnels differ? What might explain the difference?
---
The main difference I see is the conversion volumes. OTP types have a huge CR of 60% compare to 7% in Subscription and this make sense as both are very different source of revenue or alternatives for the customers

### Compare difference in traffic & conversion rates month over month
> How would we determine if conversion rate has improved?
---

**Pulling the data to answer the questions**  
First I build a query with aggregations by date, in order to have a raw table from where to calculate other metrics ⬇

In [6]:
%%sparksql

select
to_date(e.date, 'dd-MMM-yy') as event_date

,sum(sessions) as sessions
,sum(totalusers) as totalusers

,s.subscription
,s.otp

,sum(case when e.purchaseType = 'Subscription' and e.eventname= 'subcription_view' then e.totalusers end) as users_subcription_view
,sum(case when e.purchaseType = 'Subscription' and e.eventname= 'purchase' then e.totalusers end) as users_subcription_purchase

,sum(case when e.purchaseType = 'OTP' and e.eventname= 'add_to_cart' then e.totalusers end) as users_otp_view
,sum(case when e.purchaseType = 'OTP' and e.eventname= 'purchase' then e.totalusers end) as users_otp_purchase

from events_data e
left join sales_data s on to_date(e.date, 'dd-MMM-yy') = to_date(s.date, 'MM/dd/yyyy')

group by 1,4,5

only showing top 20 row(s)


event_date,sessions,totalusers,subscription,otp,users_subcription_view,users_subcription_purchase,users_otp_view,users_otp_purchase
2024-05-30,102307.0,94095.0,4667.31,16316.3,2916.0,205.0,2222.0,1352.0
2024-05-25,90311.0,83222.0,10595.4,9744.46,2538.0,142.0,1681.0,965.0
2024-04-20,110240.0,101759.0,9302.79,11988.89,2998.0,231.0,2117.0,1219.0
2024-05-19,92461.0,85657.0,10314.1,12609.94,2632.0,159.0,1873.0,1131.0
2024-04-08,116419.0,107026.0,9586.1,21018.13,913.0,162.0,2475.0,1530.0
2024-04-02,120163.0,110960.0,9036.06,20689.49,1023.0,187.0,2677.0,1673.0
2024-04-06,102045.0,94855.0,8537.47,11860.95,791.0,129.0,1951.0,1112.0
2024-04-27,105859.0,99160.0,9615.94,11844.67,1957.0,157.0,2002.0,1186.0
2024-05-05,112418.0,103674.0,9924.4,15192.96,3218.0,204.0,2686.0,1670.0
2024-05-29,105472.0,97766.0,10107.49,16265.16,3087.0,206.0,2203.0,1350.0


On top of this table I calculate the rest of the metrics where we can easily change the granularity by day, week, month ⬇

In [7]:
%%sparksql -l 30

with muse_data as (

    select
    to_date(e.date, 'dd-MMM-yy') as event_date

    ,sum(sessions) as sessions
    ,sum(totalusers) as totalusers

    ,s.subscription
    ,s.otp

    ,sum(case when e.purchaseType = 'Subscription' and e.eventname= 'subcription_view' then e.totalusers end) as users_subcription_view
    ,sum(case when e.purchaseType = 'Subscription' and e.eventname= 'purchase' then e.totalusers end) as users_subcription_purchase

    ,sum(case when e.purchaseType = 'OTP' and e.eventname= 'add_to_cart' then e.totalusers end) as users_otp_view
    ,sum(case when e.purchaseType = 'OTP' and e.eventname= 'purchase' then e.totalusers end) as users_otp_purchase

    from events_data e
    left join sales_data s on to_date(e.date, 'dd-MMM-yy') = to_date(s.date, 'MM/dd/yyyy')

    group by 1,4,5

)

select

date(date_trunc('month', event_date)) as event_month
,sum(sessions) as sessions
,sum(totalusers) as total_users
,round((sum(users_subcription_purchase) / sum(users_subcription_view))*100,2) as sub_conversion_rate
,round((sum(users_otp_purchase) / sum(users_otp_view))*100,2) as otp_conversion_rate

,round(sum(subscription),2) as subscription_sales
,round(sum(otp),2) as otp_sales
,sum(subscription) / sum(users_subcription_purchase) as avg_subscription_order_value
,sum(otp) / sum(users_otp_purchase) as avg_otp_order_value


,(sum(subscription) + sum(otp)) / sum(totalusers) as arpu_tota_users
,(sum(subscription) + sum(otp)) / (sum(users_subcription_view) + sum(users_otp_view)) as arpu__active_users

from muse_data

group by 1

order by 1 

event_month,sessions,total_users,sub_conversion_rate,otp_conversion_rate,subscription_sales,otp_sales,avg_subscription_order_value,avg_otp_order_value,arpu_tota_users,arpu__active_users
2024-04-01,3520569.0,3264714.0,9.82,61.36,270680.37,549383.18,44.65199109204884,11.790097645771185,0.2511900123563657,5.956474258403789
2024-05-01,3328433.0,3079039.0,6.8,60.06,287396.7,504725.82,44.214876923076915,11.537645041832395,0.2572629057313012,4.704625618425975


In [8]:
%%sparksql -l 30

with muse_data as (

    select
    to_date(e.date, 'dd-MMM-yy') as event_date

    ,sum(sessions) as sessions
    ,sum(totalusers) as totalusers

    ,s.subscription
    ,s.otp

    ,sum(case when e.purchaseType = 'Subscription' and e.eventname= 'subcription_view' then e.totalusers end) as users_subcription_view
    ,sum(case when e.purchaseType = 'Subscription' and e.eventname= 'purchase' then e.totalusers end) as users_subcription_purchase

    ,sum(case when e.purchaseType = 'OTP' and e.eventname= 'add_to_cart' then e.totalusers end) as users_otp_view
    ,sum(case when e.purchaseType = 'OTP' and e.eventname= 'purchase' then e.totalusers end) as users_otp_purchase

    from events_data e
    left join sales_data s on to_date(e.date, 'dd-MMM-yy') = to_date(s.date, 'MM/dd/yyyy')

    group by 1,4,5

)

select

date(date_trunc('week', event_date)) as event_month
,sum(sessions) as sessions
,sum(totalusers) as total_users
,round((sum(users_subcription_purchase) / sum(users_subcription_view))*100,2) as sub_conversion_rate
,round((sum(users_otp_purchase) / sum(users_otp_view))*100,2) as otp_conversion_rate

,round(sum(subscription),2) as subscription_sales
,round(sum(otp),2) as otp_sales
,sum(subscription) / sum(users_subcription_purchase) as avg_subscription_order_value
,sum(otp) / sum(users_otp_purchase) as avg_otp_order_value


,(sum(subscription) + sum(otp)) / sum(totalusers) as arpu_tota_users
,(sum(subscription) + sum(otp)) / (sum(users_subcription_view) + sum(users_otp_view)) as arpu_active_users

from muse_data

group by 1

order by 1 

event_month,sessions,total_users,sub_conversion_rate,otp_conversion_rate,subscription_sales,otp_sales,avg_subscription_order_value,avg_otp_order_value,arpu_tota_users,arpu_active_users
2024-04-01,796330.0,737270.0,18.69,60.57,63271.42,119055.56,52.29042975206612,11.376546583850931,0.24730014784271703,7.676279050185247
2024-04-08,803217.0,741340.0,18.41,60.97,61886.68,127245.24,55.85440433212996,12.363509521958802,0.25512169854587635,8.25939648019564
2024-04-15,848328.0,790152.0,7.71,61.0,63691.79,132219.72,37.93435973793925,11.53951125850934,0.24794154795533013,4.830760941930712
2024-04-22,827792.0,770095.0,7.53,62.1,65666.14,125647.74,40.86256378344742,11.488318551705222,0.24842893409254702,4.9127903035283245
2024-04-29,824520.0,761771.0,6.97,63.77,63395.41,133206.82,40.327868956743,11.357048341717112,0.2580857370522112,4.801031257631258
2024-05-06,762094.0,705291.0,7.03,59.14,64821.11,115226.03,42.009792611795206,11.508792449061126,0.2552806430253612,4.629651324247879
2024-05-13,752668.0,700630.0,6.7,58.45,67344.23,114756.47,46.38032369146006,11.485984385947352,0.2599099381984785,4.697433317855853
2024-05-20,718037.0,661977.0,6.64,59.09,68731.28,107367.2,50.205463842220595,12.107262065854759,0.2660190308726738,4.942560273934154
2024-05-27,516016.0,475227.0,6.88,60.99,39269.01,79384.22,38.423688845401166,12.044336216052193,0.24967695438180068,4.623333463216958


> How would we determine if conversion rate has improved?  
We can track the weekly/monthly performance trying to see if there is a positive change on the trend. Also we can try to measure incrementality of mkt campaigns as positive signs of improvement. 

### Revenue and CR Analysis

- **Traffic:** Sessions and users decrease by -5% MoM, this might be due to natural monthly fluctation (like seasonality)
- **Conversion Rate:**  
    - Subscription has a drop of 3 points and OTP seems to have an stable performance arround 60% (+/- 1.6% standard deviation)
    - Weekly Subscription CR have a negative trend, there is a drop of 11 points from Abril to end of May
- **AOV**
    - subscription: 44  
    - OTP: 11
- **ARPU**
    - It's not clear from the data if 'total_uses' are active/unique users, im calculating 2 'types' of ARPU value:
    - **ARPU total users:** 0.25 --> total sales (Sub + OTP) over Total users
    - **ARPU active users:** 5.48 --> for this exercise, Im assuming that only those users that trigger the event 'subcription_view' or 'add_to_cart' are active. This help to quantify how important is engagement with the brand/site
    - Both ARPU calculation are very consistent, daily and weekly, with low volatility

#### What would be your approach to measuring a revenue impact in A/B testing? What metrics would you use?

I belive ARPU and Conversion rate are good metrics for an A/B test. Depending what do we want to test, the approach could be causal impact analysis with observational data to try to measure incrementality or testing changes within the product that could improve/optimize the funnel or some steps on it. 

In [9]:
spark.sql("""

with muse_data as (

    select
    to_date(e.date, 'dd-MMM-yy') as event_date

    ,sum(sessions) as sessions
    ,sum(totalusers) as totalusers

    ,s.subscription
    ,s.otp

    ,sum(case when e.purchaseType = 'Subscription' and e.eventname= 'subcription_view' then e.totalusers end) as users_subcription_view
    ,sum(case when e.purchaseType = 'Subscription' and e.eventname= 'purchase' then e.totalusers end) as users_subcription_purchase

    ,sum(case when e.purchaseType = 'OTP' and e.eventname= 'add_to_cart' then e.totalusers end) as users_otp_view
    ,sum(case when e.purchaseType = 'OTP' and e.eventname= 'purchase' then e.totalusers end) as users_otp_purchase

    from events_data e
    left join sales_data s on to_date(e.date, 'dd-MMM-yy') = to_date(s.date, 'MM/dd/yyyy')

    group by 1,4,5

)

select

date_trunc('week', event_date) as event_month
,sum(sessions) as sessions
,sum(totalusers) as totalusers
,round((sum(users_subcription_purchase) / sum(users_subcription_view))*100,2) as sub_cr
,round((sum(users_otp_purchase) / sum(users_otp_view))*100,2) as otp_cr

,round(sum(subscription),2) as subscription_sales
,round(sum(otp),2) as otp_sales
,sum(subscription) / sum(users_subcription_purchase) as avg_subscription_order_value
,sum(otp) / sum(users_otp_purchase) as avg_otp_order_value


,(sum(subscription) + sum(otp)) / sum(totalusers) as arpu_tota_users
,(sum(subscription) + sum(otp)) / (sum(users_subcription_view) + sum(users_otp_view)) as arpu_active_users

from muse_data

group by 1

order by 1 

""").toPandas().describe()

,sessions,totalusers,sub_cr,otp_cr,subscription_sales,otp_sales,avg_subscription_order_value,avg_otp_order_value,arpu_tota_users,arpu_active_users
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,761000.222222,704861.444444,9.617778,60.675556,62008.563333,117123.222222,44.920988,11.696812,0.254196,5.485915
std,100676.601006,94696.050558,5.076787,1.648166,8792.085134,16560.706703,6.528534,0.370768,0.006422,1.419103
min,516016.000000,475227.000000,6.640000,58.450000,39269.010000,79384.220000,37.934360,11.357048,0.247300,4.623333
25%,752668.000000,700630.000000,6.880000,59.140000,63271.420000,114756.470000,40.327869,11.485984,0.248429,4.697433
50%,796330.000000,737270.000000,7.030000,60.970000,63691.790000,119055.560000,42.009793,11.508792,0.255122,4.830761
75%,824520.000000,761771.000000,7.710000,61.000000,65666.140000,127245.240000,50.205464,12.044336,0.258086,4.942560
max,848328.000000,790152.000000,18.690000,63.770000,68731.280000,133206.820000,55.854404,12.363510,0.266019,8.259396


In [12]:
!jupyter nbconvert --to html MUSE.ipynb \
--TagRemovePreprocessor.remove_cell_tags='{"remove_cell"}'\
--TagRemovePreprocessor.remove_all_outputs_tags='{"remove_output"}'\
--TagRemovePreprocessor.remove_input_tags='{"remove_input"}'

[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook MUSE.ipynb to html
[NbConvertApp] Writing 611091 bytes to MUSE.html
